In [1]:
import numpy as np
import pandas as pd


import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


import plotly.graph_objects as go
import matplotlib

matplotlib.use("agg")

In [2]:
series = pd.read_csv(
    "../assets/datasets/time_series_solar.csv",
    parse_dates=["Datetime"],
    index_col="Datetime",
)["Incoming Solar"]
series.tail()

Datetime
2013-09-30 19:00:00    0.0
2013-09-30 20:00:00    0.0
2013-09-30 21:00:00    0.0
2013-09-30 22:00:00    0.0
2013-09-30 23:00:00    0.0
Name: Incoming Solar, dtype: float64

In [3]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if len(data.shape) == 1 else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()

    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [("var%d(t-%d)" % (j + 1, i)) for j in range(n_vars)]

    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [("var%d(t)" % (j + 1)) for j in range(n_vars)]
        else:
            names += [("var%d(t+%d)" % (j + 1, i)) for j in range(n_vars)]

    agg = pd.concat(cols, axis=1)
    agg.columns = names

    if dropnan:
        agg.dropna(inplace=True)

    return agg

In [4]:
# Resample the data to daily frequency
series = series.resample("D").sum()

data = series_to_supervised(series, 3)
print(data)

            var1(t-3)  var1(t-2)  var1(t-1)  var1(t)
Datetime                                            
2007-10-04     1381.5     3953.2     3098.1   2213.9
2007-10-05     3953.2     3098.1     2213.9   1338.8
2007-10-06     3098.1     2213.9     1338.8   3671.5
2007-10-07     2213.9     1338.8     3671.5   4193.7
2007-10-08     1338.8     3671.5     4193.7   4213.8
...               ...        ...        ...      ...
2013-09-26     4113.6     2134.2     1250.2   1034.2
2013-09-27     2134.2     1250.2     1034.2   2182.3
2013-09-28     1250.2     1034.2     2182.3   3384.5
2013-09-29     1034.2     2182.3     3384.5    478.2
2013-09-30     2182.3     3384.5      478.2   2554.8

[2189 rows x 4 columns]


In [5]:
scaler = MinMaxScaler(feature_range=(-1, 1))
train, test = train_test_split(data, test_size=0.2, shuffle=False)
train = scaler.fit_transform(train)
test = scaler.transform(test)

X_train, y_train = train[:, :-1], train[:, -1]
X_test, y_test = test[:, :-1], test[:, -1]

X_train = torch.from_numpy(X_train).type(torch.Tensor)
X_test = torch.from_numpy(X_test).type(torch.Tensor)
y_train = torch.from_numpy(y_train).type(torch.Tensor).view(-1)
y_test = torch.from_numpy(y_test).type(torch.Tensor).view(-1)

In [6]:
data.shape, train.shape, test.shape

((2189, 4), (1751, 4), (438, 4))

In [7]:
class FeedForwardNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.activation = nn.ReLU()

    def forward(self, x):
        out = self.activation(self.fc1(x))
        out = self.fc2(out)
        return out

In [8]:
model = FeedForwardNN(input_dim=X_train.shape[1], hidden_dim=32, output_dim=1)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

epochs = 200

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    out = model(X_train).reshape(
        -1,
    )
    loss = loss_fn(out, y_train)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, Loss: {loss.item()}")


model.eval()
y_pred = model(X_test).reshape(
    -1,
)
test_loss = loss_fn(y_pred, y_test)
print(f"Test Loss: {test_loss.item()}")

Epoch: 0, Loss: 0.3756842017173767
Epoch: 10, Loss: 0.1168794333934784
Epoch: 20, Loss: 0.10837151855230331
Epoch: 30, Loss: 0.10781492292881012
Epoch: 40, Loss: 0.10598704218864441
Epoch: 50, Loss: 0.10474874824285507
Epoch: 60, Loss: 0.10420942306518555
Epoch: 70, Loss: 0.10370013117790222
Epoch: 80, Loss: 0.10328510403633118
Epoch: 90, Loss: 0.10292850434780121
Epoch: 100, Loss: 0.10263549536466599
Epoch: 110, Loss: 0.10238544642925262
Epoch: 120, Loss: 0.10213649272918701
Epoch: 130, Loss: 0.10188670456409454
Epoch: 140, Loss: 0.10168652981519699
Epoch: 150, Loss: 0.10153103619813919
Epoch: 160, Loss: 0.10142461210489273
Epoch: 170, Loss: 0.10132542997598648
Epoch: 180, Loss: 0.10122353583574295
Epoch: 190, Loss: 0.10112378001213074
Test Loss: 0.09318147599697113


In [9]:
X_test

tensor([[ 0.7575,  0.7865,  0.4256],
        [ 0.7865,  0.4256,  0.7273],
        [ 0.4256,  0.7273,  0.7913],
        ...,
        [-0.6711, -0.7283, -0.4242],
        [-0.7283, -0.4242, -0.1058],
        [-0.4242, -0.1058, -0.8756]])

In [10]:
X_test.detach().numpy().shape, y_pred.detach().numpy().shape

((438, 3), (438,))

In [11]:
np.concatenate(
    [X_test.detach().numpy(), y_pred.detach().numpy().reshape(-1, 1)], axis=1
)

array([[ 0.75748587,  0.786489  ,  0.4256049 ,  0.4687724 ],
       [ 0.786489  ,  0.4256049 ,  0.72734374,  0.59172523],
       [ 0.4256049 ,  0.72734374,  0.79130965,  0.6041649 ],
       ...,
       [-0.67111206, -0.72832376, -0.42422757, -0.49590266],
       [-0.72832376, -0.42422757, -0.10580195, -0.22286111],
       [-0.42422757, -0.10580195, -0.875591  , -0.49233913]],
      dtype=float32)

In [12]:
pred = scaler.inverse_transform(
    np.concatenate(
        [X_test.detach().numpy(), y_pred.detach().numpy().reshape(-1, 1)], axis=1
    )
)
pred

array([[6643.8    , 6753.3    , 5390.8003 , 5553.777  ],
       [6753.3    , 5390.8003 , 6530.     , 6017.979  ],
       [5390.8003 , 6530.     , 6771.5005 , 6064.945  ],
       ...,
       [1250.2    , 1034.2001 , 2182.3    , 1911.6943 ],
       [1034.2001 , 2182.3    , 3384.5    , 2942.549  ],
       [2182.3    , 3384.5    ,  478.20004, 1925.1482 ]], dtype=float32)

In [13]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=data.index[: train.shape[0]],
        y=data.iloc[: train.shape[0]]["var1(t)"],
        name="Incoming Solar",
    )
)
fig.add_trace(
    go.Scatter(x=data.index[train.shape[0] :], y=pred[:, -1], name="Prediction")
)

fig.show()